## Оглавление 

* [Анализ данных и визуализация информации](#1)

	* [Подготовка данных](#1-1)

* [Разведывательный анализ данных EDA](#2)

	* [Общая информация](#2-1)

	* [Гистограммы](#2-2)

	* [Визуализация средствами pandas](#2-3)

	* [Boxplot](#2-4)

	* [Парные матрицы рассеяния](#2-5)

	* [Корреляция](#2-6)

	* [Вариация ](#2-7)

	* [Группировки](#2-8)

* [Обработка выбросов](#3)

	* [Удаляем выбросы](#3-1)

* [Нормирование данные](#4)

	* [MinMaxScaler](#4-1)

	* [StandardScaler](#4-2)

	* [RobustScaler](#4-3)

	* [Различия в способах нормирования](#4-4)

* [Кластеризация](#5)

* [Кластеризация методом Kmeans](#6)

	* [Метод локтя](#6-1)

	* [Метод Коэффициента Силуэта:](#6-2)

	* [Дешифровка анализа](#6-3)

	* [Визуализация результатов анализа](#6-4)

* [DBSCAN](#7)

* [Ассоциативные правила](#8)



# Анализ данных и визуализация информации<a class="anchor" id="1"></a>

## Подготовка данных<a class="anchor" id="1-1"></a>

Ниже приведен код, который упрощает повторную обработку датасета. Данный набор данных мы будем использовать дальше в учебных целях.

**Необходимо присвоить переменной `sql_pass` значение пароля для подключения к базе данных**

```python
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
import sqlalchemy

engine = sqlalchemy.create_engine(
                "mysql+pymysql://root:%s@159.69.219.206:3307/rzd" %(sql_pass), encoding='utf8', convert_unicode=True
            )

with engine.connect() as session:
    df=pd.read_sql('SELECT * FROM ku_asrb LIMIT 1000', con=session)

columns_date=['Дата события', 'Дата создания события в системе AC PБ', ]
columns_float=[ 'Размер возмещенного ущерба (тыс.руб.)',
               'Итоговый суммарный ущерб (тыс.руб.)']
columns_int=['Погибло всего', 'Погибло пассажиров',
       'Погибло сторонних', 'Погибло прочих',
       'Получили тяжкие телесные повреждения: сотрудников ОАО "РЖД"',
       'Получили тяжкие телесные повреждения: пассажиров',
       'Получили тяжкие телесные повреждения: сторонние',
       'Получили тяжкие телесные повреждения: прочие', 'Ранено легко', 
       'Путь общего/необщего пользования',
       'Номер пути', 'Километр', 'Пикет', 'Общее время задержки']
columns_time=['Время полного перерыва движения', 'Время расстройства маневровой работы',
              'Количество задержанных поездов']

def ddate(s):
    try:
        res=pd.to_datetime(s, format='%d%b%Y:%H:%M:%S')
    except:
        res=np.NaN
    return res

for i in columns_date:
    df[i]=df[i].apply(lambda x: ddate(x))
    

for i in columns_float:
    df.loc[df[i]=='.', i]='0'
    df[i]=df[i].astype(float)
    
for i in columns_int:
    df[i].fillna('.', inplace=True)
    df.loc[df[i]=='.', i]='0'
    df[i]=df[i].astype(int)
    
def time_to_sec(s):
    s3=s.split(':')
    if len(s3)<3:
        return 0
    else:
        return int(s3[0])*60*60+int(s3[1])*60+int(s3[2])
    
for i in columns_time:
    df[i]=df[i].apply(lambda x: time_to_sec(x))
    
df['Время до регистрации']=df.apply(lambda x:(x['Дата создания события в системе AC PБ']-x['Дата события']).total_seconds(), axis=1)

df.info()
```

# Разведывательный анализ данных EDA<a class="anchor" id="2"></a>

Разведочный анализ данных (англ. exploratory data analysis, EDA) — анализ основных свойств данных, нахождение в них общих закономерностей, распределений и аномалий, построение начальных моделей, зачастую с использованием инструментов визуализации.

## Общая информация<a class="anchor" id="2-1"></a>

Выберем несколько колонок для примера

```python
col=['Итоговый суммарный ущерб (тыс.руб.)', 'Общее время задержки',
       'Количество задержанных поездов', 'Время до регистрации']
df[col].describe()
```

## Гистограммы<a class="anchor" id="2-2"></a>

Построим гистограммы.

```python
df[col].hist(figsize=(15,5), bins=40);
```

Выведем события выше некоторого уровня, чтобы более внимательно их изучить.

```python
df[df['Общее время задержки']>30]
```

Строки с большими значениями (выбросы), могут мешать рассмотреть тенденции в основном массиве данных. Их стоит часто удалять или заменить на более приемлемые значения.

## Визуализация средствами pandas<a class="anchor" id="2-3"></a>

Расширим наши знания о построении графиков и визуализации информации.

- `color` - определяет цвет графика.
- `legend=True` - разрешает вывод легенды графика. Значение False подавит вывод.
- `.set_ylabel`- задает подпись оси Y
- `.set_xlabel` - задает подпись оси X
- `.set_title` - задает заголовок
- `figsize=(15,5)` - задает размер графика по ширине и высоте

```python
ax=df[df['Общее время задержки']<15]['Общее время задержки'].hist(figsize=(15,5), bins=40,  color='r');
ax.set_ylabel('Частота')
ax.set_title('Общее время задержки');
```

Ниже мы просуммируем все задержки по времени по месяцам и построим график. Изначально мы создадим копию DataFrame и установим индекс дату события методом `.set_index()`. Для выполнения суммирования мы будем использовать метод `.resample()`. Где параметр:
- `W` - неделя
- `M` - месяц
- `Q` - квартал
- `Y` - год.

```python
dfM=df[['Общее время задержки', 'Дата события']].copy()
dfM=dfM.set_index('Дата события')
dfM=dfM.resample('M').sum()
ax=dfM.plot(legend=False, figsize=(15,4), color='r')
ax.set_ylabel('Время')
ax.set_title('Общее время задержки');
```

Вы можете также строить и другие графики:
- `bar` - столбчатая диаграмма.
- `hist` - гистограмма.
- `box` - ящик с усами.
- `kde` или `density` - график плотности
- `scatter` - матрицы рассеяния
- `hexbin` - альтернатива диаграммам рассеяния
- `pie` - круговая диаграмма.

Подробнее и примеры по [ссылке](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html).

Пример

```python
dfM.plot.bar(legend=False, figsize=(15,4), color='r');
ax.set_ylabel('Время')
ax.set_title('Общее время задержки');
```

## Boxplot<a class="anchor" id="2-4"></a>

Диаграммы размаха («ящик с усами») (Box and Whisker Plot или Box Plot) – это удобный способ визуального представления групп числовых данных через квартили.

Прямые линии, исходящие из ящика, называются «усами» и используются для обозначения степени разброса (дисперсии) за пределами верхнего и нижнего квартилей. Выбросы иногда отображаются в виде отдельных точек, находящихся на одной линии с усами. Диаграммы размаха могут располагаться как горизонтально, так и вертикально.

![Ящик с усами](https://3.bp.blogspot.com/-sqSGopnp0lo/Uvu_wl_dPQI/AAAAAAAAAgs/F2DBOSdfiU4/s1600/boxplot.PNG)

Виды наблюдений, которые можно сделать на основе ящика с усами:
- Каковы ключевые значения, например: средний показатель, медиана 25го перцентиля и так далее.
- Существуют ли выбросы и каковы их значения.
- Симметричны ли данные.
- Насколько плотно сгруппированы данные.
- Смещены ли данные и, если да, то в каком направлении.

```python
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,10))
sns.boxplot(data=df[col], palette='rainbow', orient='h');
```

Удобнее рассматривать не нормированные данные на отдельных графиках.

```python
sns.boxplot(data=df[df['Время до регистрации']<100000]['Время до регистрации'], palette='rainbow', orient='h');
```

## Парные матрицы рассеяния<a class="anchor" id="2-5"></a>

```python
sns.pairplot(df[col],height=3);
```

Укрупненный срез.

```python
sns.jointplot(x='Общее время задержки', y='Итоговый суммарный ущерб (тыс.руб.)', data=df, kind='scatter');
```

## Корреляция<a class="anchor" id="2-6"></a>

При разведочном анализе стоит посмотреть на взаимосвязь между переменными. А именно, на корреляцию.

```python
df[col].corr()
```

И построить тепловую карту для лучшего восприятия.

```python
corr = df[col].corr()
plt.figure(figsize=(14, 14))
sns.heatmap(corr[(corr >= 0.3) | (corr <= -0.3)],
            cmap="RdBu_r", vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);
```

Вариант тепловой карты с группировкой по наиболее тесной взаимосвязи.

```python
plt.figure(figsize=(14,14))
sns.clustermap(df[col].corr())
```

## Вариация <a class="anchor" id="2-7"></a>

Помимо корреляции надо обращать внимание и на размах величины, а именно на ее коэффициент вариации.

```python
from scipy.stats import variation
pd.DataFrame(variation(df[col]), index=col)
````

Более наглядно об идее.

```python
df2=(df[col]-df[col].min())/(df[col].max()-df[col].min())
df2.plot(figsize=(17,5))
```

## Группировки<a class="anchor" id="2-8"></a>

При формировании гипотез полезно посмотреть и на группировки по различным качественным признакам, понять как они влияют на показатели.

```python
df.groupby('Код дороги')[col].mean()
```

# Обработка выбросов<a class="anchor" id="3"></a>

Давайте вернемся к выбросам. Мы увидели, что они очень сильно портят восприятие данных аналитиком. 

Выбросы - это значения, которые намного превышают следующие ближайшие точки данных.

Есть два типа выбросов:
- Одномерные выбросы: Одномерные выбросы - это точки данных, значения которых выходят за пределы диапазона ожидаемых значений, основанных на одной переменной.
- Многовариантные выбросы: при построении данных некоторые значения одной переменной могут не выходить за пределы ожидаемого диапазона, но при отображении данных с какой-либо другой переменной эти значения могут находиться далеко от ожидаемого значения.

Аналитик основываясь на своем опыте должен принять решение, как поступить с выбросами. Заменить, удалить или оставить. В нашем случае, мы удалим выбросы.

Следующие операции выполним последовательно.

```python
df[col].describe()

df['Итоговый суммарный ущерб (тыс.руб.)'].hist(figsize=(14,4), bins=50)

sns.boxplot(df[df['Итоговый суммарный ущерб (тыс.руб.)']<50]['Итоговый суммарный ущерб (тыс.руб.)']);

df[df['Итоговый суммарный ущерб (тыс.руб.)']<50]['Итоговый суммарный ущерб (тыс.руб.)'].hist(figsize=(14,4), bins=50);

df.drop(df[df['Итоговый суммарный ущерб (тыс.руб.)']>50].index, inplace=True, axis='index')
```
Проделаем такие же операции и с другими параметрами. Но, например, в случае общего времени задержки, всем выбросам присвоим некоторое максимальное значение.

## Удаляем выбросы

На всякий случай удаление ключевых выбросов собрано в одном месте.

```python
df.drop(np.where(df['Итоговый суммарный ущерб (тыс.руб.)']>30000)[0], inplace=True)
df=df[df['Общее время задержки']<18]
df=df[df['Количество задержанных поездов']<73000]
df=df[df['Время до регистрации']<100000]
```

# Нормирование данные<a class="anchor" id="3-1"></a>

Для большинства алгоритмов машинного обучения надо выполнить нормирование данные. Нормирование данных заключается в приведении диапазона изменения значений признаков к некоторым требуемым границам (например, от 0 до 1). Нормирование является необходимым начальным этапом обработки данных при использовании многих многомерных статистических методов — снижения размерности признакового пространства, классификации и т.д., особенно если переменные измерены в шкалах, существенно различающихся в величинах (например, от миллиметров до километров).

Воспользуемся препроцессиногом библиотеки sklearn.

```python
from sklearn import preprocessing

import random
import numpy as np
import matplotlib.pyplot as plt

a=np.array([[random.randint(0,10), random.randint(0,30)] for z in range(20)])
a
```

## MinMaxScaler<a class="anchor" id="4-1"></a>

MinMaxScaler лучше всего подходит для отображения данных и их использования при той же кластеризации.

```python
min_max_scaler = preprocessing.MinMaxScaler()
minmax = min_max_scaler.fit_transform(a)
minmax
```

## StandardScaler<a class="anchor" id="4-2"></a>

StandardScaler лучше подходит для нормирования в рамках алгоритмов машинного обучения.

```python
scaler = preprocessing.StandardScaler().fit(a)
standart=scaler.fit_transform(a)
standart
```

## RobustScaler<a class="anchor" id="4-3"></a>

RobustScaler лучше работает с зашумленными данными и выбросами.

```python
from sklearn.preprocessing import RobustScaler
robust = RobustScaler().fit_transform(a)
robust
```

## Различия в способах нормирования<a class="anchor" id="4-4"></a>

Для того, чтобы лучше понять, как работают различные способы нормирования, отобразим их на графике.

```python
import seaborn as sns
import matplotlib.pyplot as plt

f=np.array([a, standart, minmax, robust])
label=['Оригинальные', 'Standart', 'MinMax', 'Robust']

fig, axs = plt.subplots(1, 4, figsize=(15, 4))
plt.title("title")
for i, ax in enumerate(axs):
    x=f[i,:,0]
    y=f[i,:,1]
    ax.title.set_text(label[i])
    ax.scatter(x, y)
    ax.grid()
```

Или более наглядно.

```python
import seaborn as sns

m=[]

for i in range(len(f)):
    for j in f[i]:
        m.append([j[0], j[1], label[i]])
dfG=pd.DataFrame(m, columns=['x','y','standart'])

fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(x='x', y='y', hue='standart', data=dfG) 
plt.show()
```

# Кластеризация<a class="anchor" id="5"></a>

Обучение без учителя (unsupervised learning, неконтролируемое обучение) – класс методов машинного обучения для поиска шаблонов в наборе данных. Данные, получаемые на вход таких алгоритмов, обычно не размечены, то есть передаются только входные переменные X без соответствующих меток Y. Если в контролируемом обучении (обучении с учителем, supervised learning) система пытается извлечь уроки из предыдущих примеров, то в обучении без учителя система старается самостоятельно найти шаблоны непосредственно из приведенного примера.

Методы кластеризации данных являются одним из наиболее популярных семейств машинного обучения без учителя. Рассмотрим некоторые из них подробнее. 

![Классическая картинка](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png)


[Описание алгоритмов](https://scikit-learn.org/stable/modules/clustering.html)

# Кластеризация методом Kmeans<a class="anchor" id="6"></a>

Иерархическая кластеризация хуже подходит для кластеризации больших объемов данных в сравнении с методом k-средних. Это объясняется тем, что временная сложность алгоритма линейна для метода k-средних (O(n)) и квадратична для метода иерархической кластеризации (O(n2)).

В кластеризации при помощи метода k-средних алгоритм начинает построение с произвольного выбора начальных точек, поэтому результаты, генерируемые при многократном запуске алгоритма, могут отличаться. В то же время в случае иерархической кластеризации результаты воспроизводимы.

Из центроидной геометрии построения метода k-средних следует, что метод хорошо работает, когда форма кластеров является гиперсферической (например, круг в 2D или сфера в 3D).

Метод k-средних более чувствителен к зашумленным данным, чем иерархический метод.

## Метод локтя<a class="anchor" id="6-1"></a>

Если истинная метка заранее не известна(как в вашем случае), то K-Means clustering можно оценить с помощью критерия локтя или коэффициента силуэта.

Идея метода локтя состоит в том , чтобы выполнить кластеризацию k-средних по заданному набору данных для диапазона значений k ( num_clusters, например k=1-10) и для каждого значения k вычислить сумму квадратов ошибок (SSE).

После этого постройте линейный график SSE для каждого значения k. Мы хотим свести к минимуму SSE. SSE имеет тенденцию уменьшаться к 0, когда мы увеличиваем k (и SSE равно 0, когда k равно числу точек данных в наборе данных, потому что тогда каждая точка данных является своим собственным кластером, и нет никакой ошибки между ней и центром ее кластера).

Таким образом, цель состоит в том, чтобы выбрать наименьшее значение k , который все еще имеет низкий SSE, и локоть обычно представляет, где мы начинаем иметь убывающую отдачу при увеличении k.

```python
from sklearn.cluster import KMeans

X=df[col].values ### переменую после нормирования

sse = {}
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Количество кластеров")
plt.ylabel("SSE")
plt.show()
```

## Метод Коэффициента Силуэта<a class="anchor" id="6-2"></a>

Более высокая оценка коэффициента силуэта относится к модели с более четко определенными кластерами. Коэффициент «силуэт» вычисляется с помощью среднего внутрикластерного расстояния (a) и среднего расстояния до ближайшего кластера (b) по каждому образцу. Силуэт вычисляется как (b - a) / max(a, b).  b — это расстояние между a и ближайшим кластером, в который a не входит. Можно вычислить среднее значение силуэта по всем образцам и использовать его как метрику для оценки количества кластеров.

Более высокий коэффициент силуэта указывает на то, что объект хорошо подобран к своему собственному кластеру и плохо подобран к соседним кластерам.

```python
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster).fit(X)
    label = kmeans.labels_
    sil_coeff = silhouette_score(X, label, metric='euclidean')
    print("Для n_clusters={}, коэффициент силуэта {}".format(n_cluster, sil_coeff))
```

## K-means

Пример выполнения кластеризации и визуализации с отображением центров кластеров.

```python
kmeans = KMeans(n_clusters = 4).fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c = y_kmeans, s = 20, cmap = 'summer')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c = 'blue', s = 100, alpha = 0.9);
plt.show()
```

Обработка результатов происходит также как и в случае иерархической кластеризации. 

## Дешифровка анализа<a class="anchor" id="6-3"></a>

Попробуем объяснить кластера. Создадим копию DataFrame и сформируем столбец с номерами кластеров.

```python
df_result=df[col].copy()
df_result['kmeans']=kmeans.labels_
df_result.sample(10)
```

Сгруппируем значения вокруг кластеров. 

```python
df_analize=df_result.groupby('kmeans')[col].mean()
df_analize['Count']=df_result.groupby('kmeans')['kmeans'].count()
df_analize
```

## Визуализация результатов анализа<a class="anchor" id="6-4"></a>

Построим диаграмму рассеяния.

```python
import seaborn as sns
import matplotlib.pyplot as plt

x=col[2] #Изменяйте столбцы 
y=col[3]
print(x,y)
sns.lmplot( x=x, y=y, data=df_result, fit_reg=False, hue='kmeans', legend=False)
plt.legend(loc='lower right')
plt.show()
```

Тоже самое, но каждый график отдельно по кластеру.

```python
x=col[1] #Изменяйте столбцы 
y=col[2]
g = sns.lmplot(x=x, y=y, hue="kmeans", col="kmeans",
               data=df_result, height=6, aspect=.4, x_jitter=.1)
```

Укрупним графики.

```python
g = sns.lmplot(x=x, y=y, hue="kmeans", col="kmeans",
               data=df_result, col_wrap=2, height=4)
```

# DBSCAN<a class="anchor" id="7"></a>

DBSCAN (Density-based spatial clustering of applications with noise, плотностной алгоритм пространственной кластеризации с присутствием шума), как следует из названия, оперирует плотностью данных. На вход он просит уже знакомую матрицу близости и два параметра — радиус -окрестности и количество соседей. 

Принцип работы метода иллюстрируется анимированной картинкой.

![DBSCAN](https://ichi.pro/assets/images/max/724/0*PQTm0HWuxQHuxHyl.gif)


[Подробнее](https://habr.com/ru/post/322034/)

Реализация алгоритма крайне проста с использованием библиотек. Но надо указать два параметра, минимальное количество объектов в кластере (определяем , опреедляем уровень шума) и эпсилон(eps) - радиус вокруг точки для определения соседа в кластере. 

Используем для расчета метод колена. Цель состоит в том, чтобы найти среднее расстояние для каждой точки до ее K ближайших соседей и выбрать расстояние, на котором происходит максимальная кривизна или резкое изменение кривой на графике.

```python
X=df[col].values ### переменую после нормирования
X = preprocessing.StandardScaler().fit_transform(X)

from sklearn.neighbors import NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=5)
nearest_neighbors.fit(X)
distances, indices = nearest_neighbors.kneighbors(X)
distances = np.sort(distances, axis=0)[:, 1]
#print(distances)
plt.plot(distances)
plt.show()
```

Выполним саму кластеризацию.

```python
from sklearn.cluster import DBSCAN

#eps - радиус
#минимальное количество участников в кластере
db = DBSCAN(eps=0.65, min_samples=5).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
db.labels_
```

Отрисуем график.

```python
colors = ['royalblue', 'maroon', 'forestgreen', 'mediumorchid', 'tan', 'deeppink', 'olive', 'goldenrod', 'lightcyan', 'navy']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])
plt.scatter(X[:,0], X[:,2], c=vectorizer(db.labels_))
```

Чтобы получить количество кластеров, надо выполнить `db.labels_.max()+1`.

Обработка результатов проходит аналогично другим видам кластеризации. 

# Ассоциативные правила<a class="anchor" id="8"></a>

Ассоциативные правила позволяют находить закономерности между связанными событиями. Примером такой закономерности служит правило, указывающее, что из события X следует событие Y с некоторой вероятностью. Установление таких зависимостей дает возможность находить очень простые и интуитивно понятные правила.

```python
import sqlalchemy

engine = sqlalchemy.create_engine(
                "mysql+pymysql://root:%s@159.69.219.206:3307/rzd" %(sql_pass), encoding='utf8', convert_unicode=True
            )

#прочитаем таблицу
with engine.connect() as session:
    df=pd.read_sql('SELECT * FROM grdp LIMIT 1000', con=session)
```

Проведем подготовительные работы.

```python
#выделю колонки, которые хочу проанализировать
col=['Продолжительность', 'Характер', 'Телеграмма', 'Деффект']
#удалим все значение None
df.dropna(subset=col, axis='index', inplace=True)
#выполним приведение типа к строковому
df['Продолжительность']=df['Продолжительность'].astype(str)
#сформирую список событий
transactions=df[col].values
```

Выполним сам расчет.

```python
#загрузим пакеты, необходимые для выполнения анализа
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_as = pd.DataFrame(te_ary, columns=te.columns_)
#параметры можно регулировать, например, сразу отсечь все мало встречаемое
frequent_itemsets = apriori(df_as, min_support=0.2, use_colnames=True)
```

И выведем результаты c фильтром по уровню поддержки

```python
#сгенерируем ассоциативные правила с уровнем доверия 0.1 Стоит учитывать, что данный уровень поддержки крайне
#низкий и используется только для примера
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
```

С фильтром по уровню независимости. 

```python
#найдем ассоциативные правила с уровнем независимости больше 1.1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.1)
rules
```

Расшифровка важных для нас параметров:
- antecedents - посыл
- consequents - последствия
- antecedent support - встречаемость посыла
- consequent support - встречаемость последствия
- support - совместная встречаемость
- confidence - вероятность появления последствия при наличии посыла
- lift - условно мера случайности. Если значение меньше единицы, правилу доверять не стоит
- leverage - это разность между наблюдаемой частотой, с которой условие и
следствие появляются совместно (т.е., поддержкой ассоциации), и
произведением частот появления (поддержек) условия и следствия по
отдельности. 
- Conviction - В общем виде Conviction — это «частотность ошибок» нашего правила. Должно быть больше 1.